In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df[['koi_period','koi_period_err1','koi_period_err2','koi_time0bk', 'koi_time0bk_err1','koi_time0bk_err2','koi_impact','koi_impact_err1','koi_impact_err2', 'koi_duration', 'koi_duration_err1','koi_duration_err2','koi_depth','koi_depth_err1','koi_depth_err2','koi_prad','koi_prad_err1','koi_prad_err2','koi_teq','koi_insol','koi_insol_err1','koi_insol_err2','koi_model_snr','koi_steff','koi_steff_err1','koi_steff_err2','koi_slogg','koi_slogg_err1','koi_slogg_err2','koi_srad','koi_srad_err1','koi_srad_err2','ra','dec','koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = df['koi_disposition']

In [5]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
X_train.head()

,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,0.150,0.305,-0.150,3.61600,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,0.291,0.193,-0.291,2.30900,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,0.970,0.879,-0.942,79.89690,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,0.300,0.145,-0.300,2.63120,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,0.831,0.016,-0.004,2.22739,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [9]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
from tensorflow.keras.utils import to_categorical
# One-hot encoding
#y_train_categorical = to_categorical(y_train)
#y_test_categorical = to_categorical(y_test)

# Train the Model



In [11]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=35))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1800      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 1,953
Trainable params: 1,953
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
5243/5243 - 0s - loss: 0.8666 - acc: 0.6084
Epoch 2/60
5243/5243 - 0s - loss: 0.7207 - acc: 0.6805
Epoch 3/60
5243/5243 - 0s - loss: 0.6714 - acc: 0.7107
Epoch 4/60
5243/5243 - 0s - loss: 0.6437 - acc: 0.7238
Epoch 5/60
5243/5243 - 0s - loss: 0.6253 - acc: 0.7337
Epoch 6/60
5243/5243 - 0s - loss: 0.6103 - acc: 0.7406
Epoch 7/60
5243/5243 - 0s - loss: 0.5992 - acc: 0.7431
Epoch 8/60
5243/5243 - 0s - loss: 0.5912 - acc: 0.7486
Epoch 9/60
5243/5243 - 0s - loss: 0.5805 - acc: 0.7515
Epoch 10/60
5243/5243 - 0s - loss: 0.5732 - acc: 0.7597
Epoch 11/60
5243/5243 - 0s - loss: 0.5661 - acc: 0.7608
Epoch 12/60
5243/5243 - 0s - loss: 0.5602 - acc: 0.7601
Epoch 13/60
5243/5243 - 0s - loss: 0.5561 - acc: 0.7635
Epoch 14/60
5243/5243 - 0s - loss: 0.5494 - acc: 0.7652
Epoch 15/60
5243/5243 - 0s - loss: 0.5455 - acc: 0.7696
Epoch 16/60
5243/5243 - 0s - loss: 0.5403 - acc: 0.7706
Epoch 17/60
5243/5243 - 0s - loss: 0.5383 - acc: 0.7711
Epoch 18/60
5243/5243 - 0s - loss: 0.5343 - acc: 0.7742
E

In [15]:
model_loss, model_accuracy = model.evaluate(
    X_train_scaled, y_train_categorical, verbose=2)
print(
    f"Train Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

5243/5243 - 0s - loss: 0.4696 - acc: 0.8028
Train Neural Network - Loss: 0.4695770699582708, Accuracy: 0.8027846813201904


In [16]:
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=35))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               3600      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 14,003
Trainable params: 14,003
Non-trainable params: 0
_________________________________________________________________


In [17]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_test_scaled,
    y_test_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
1748/1748 - 0s - loss: 0.8641 - acc: 0.6196
Epoch 2/100
1748/1748 - 0s - loss: 0.7304 - acc: 0.6802
Epoch 3/100
1748/1748 - 0s - loss: 0.6739 - acc: 0.7054
Epoch 4/100
1748/1748 - 0s - loss: 0.6343 - acc: 0.7391
Epoch 5/100
1748/1748 - 0s - loss: 0.6059 - acc: 0.7466
Epoch 6/100
1748/1748 - 0s - loss: 0.5856 - acc: 0.7546
Epoch 7/100
1748/1748 - 0s - loss: 0.5651 - acc: 0.7683
Epoch 8/100
1748/1748 - 0s - loss: 0.5515 - acc: 0.7792
Epoch 9/100
1748/1748 - 0s - loss: 0.5386 - acc: 0.7826
Epoch 10/100
1748/1748 - 0s - loss: 0.5255 - acc: 0.7855
Epoch 11/100
1748/1748 - 0s - loss: 0.5129 - acc: 0.7832
Epoch 12/100
1748/1748 - 0s - loss: 0.5090 - acc: 0.8015
Epoch 13/100
1748/1748 - 0s - loss: 0.4955 - acc: 0.7923
Epoch 14/100
1748/1748 - 0s - loss: 0.4833 - acc: 0.8066
Epoch 15/100
1748/1748 - 0s - loss: 0.4723 - acc: 0.8038
Epoch 16/100
1748/1748 - 0s - loss: 0.4636 - acc: 0.8164
Epoch 17/100
1748/1748 - 0s - loss: 0.4558 - acc: 0.8204
Epoch 18/100
1748/1748 - 0s - loss: 0.45

In [18]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Test Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
#print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

1748/1748 - 0s - loss: 0.1186 - acc: 0.9691
Test Neural Network - Loss: 0.11860658767157889, Accuracy: 0.9691075682640076


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [19]:
# Create the GridSearchCV model
# Support vector machine linear classifier
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [25]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.0005, 0.001]}
#grid = GridSearchCV(model2, param_grid, verbose=2, cv = 5)
grid = GridSearchCV(deep_model, param_grid, verbose=2, cv = 5, scoring='accuracy')
grid

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=<tensorflow.python.keras.engine.sequential.Sequential object at 0x00000220ABDF2AC8>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.0005, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [27]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

TypeError: Cannot clone object '<tensorflow.python.keras.engine.sequential.Sequential object at 0x00000220ABDF2AC8>' (type <class 'tensorflow.python.keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

In [22]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 0.0001}
0.7419416364676712


# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'model3.sav'
#joblib.dump(model1, filename)

In [ ]:
#This second neural network model is more reliable since has a higher accuracy ~97% (91% first model) while minimizing the loss to ~11% (vs 25%);
#I infer that this improved results are yield given that were included more parameters from the data (errors)